In [ ]:
first = True
use_mm = False

In [ ]:
import sys

In [ ]:
%%capture
if use_mm:    
    # pytorch version compatible with openmmlab
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl
    # torchvision for yolov5
    !pip install ../input/pytorch-190/torchvision-0.10.0+cu111-cp37-cp37m-linux_x86_64.whl
    # mmcv install
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/yapf-0.32.0-py2.py3-none-any.whl
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/addict-2.4.0-py3-none-any.whl
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/mmcv_full-1.4.2-cp37-cp37m-manylinux1_x86_64.whl

    # mmdetection install
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/pycocotools-2.0.3/pycocotools-2.0.3.tar
    !pip install ../input/openmmlab-essential-repositories/openmmlab-repos/src/terminaltables-3.1.10-py2.py3-none-any.whl

    !cp -r ../input/openmmlab-essential-repositories/openmmlab-repos/mmdetection /kaggle/working/
    %cd /kaggle/working/mmdetection
    !pip install -e .
    %cd ..
    
    sys.path.append('./mmdetection')

In [ ]:
%%capture
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

In [ ]:
%%capture
# https://www.kaggle.com/c/severstal-steel-defect-detection/discussion/109679
!mkdir -p /tmp/pip/cache2/
!cp /kaggle/input/yolov5-package/sahi-0.8.22-py3-none-any.whl /tmp/pip/cache2/
!cp /kaggle/input/yolov5-package/terminaltables-3.1.10-py2.py3-none-any.whl /tmp/pip/cache2/
!cp /kaggle/input/yolov5-package/thop-0.0.31.post2005241907-py3-none-any.whl /tmp/pip/cache2/
!cp /kaggle/input/yolov5-package/yolov5-6.0.6-py36.py37.py38-none-any.whl /tmp/pip/cache2/
!cp /kaggle/input/yolov5-package/fire-0.4.0.xyz /tmp/pip/cache2/fire-0.4.0.tar.gz
!pip install --no-index --find-links /tmp/pip/cache2/ yolov5
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

!mkdir -p /tmp/pip/cache/
!cp /kaggle/input/sahi-0822/sahi-0.8.22-py3-none-any.whl /tmp/pip/cache/
!cp /kaggle/input/sahi-0822/terminaltables-3.1.10-py2.py3-none-any.whl /tmp/pip/cache/
!cp /kaggle/input/sahi-0822/fire-0.4.0.xyz /tmp/pip/cache/fire-0.4.0.tar.gz
!pip install --no-index --find-links /tmp/pip/cache/ sahi

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
%cd /kaggle/working/

In [ ]:
from pathlib import Path
import cv2
from tqdm import tqdm
import pandas as pd
import numpy as np
import ast
import sys
sys.path.append('/kaggle/input/weightedboxesfusion/')
sys.path.append('/kaggle/input/pickle5/pickle5-backport-master')
import pickle5 as pickle
import tgbr_util as util
import torch
import yaml
from  glob import glob 
if use_mm:
    from mmdet.apis import init_detector, inference_detector
    import mmcv

from sahi.model import Yolov5DetectionModel
from sahi.predict import get_sliced_prediction

In [ ]:
sys.path.append('/kaggle/input/raft-pytorch')
from raft.core.raft import RAFT
#from raft.core.utils import flow_viz
from raft.core.utils.utils import InputPadder
from raft.config import RAFTConfig

raft_config = RAFTConfig(
    dropout=0,
    alternate_corr=False,
    small=False,
    mixed_precision=True
)

raft_model = RAFT(raft_config)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

weights_path = '/kaggle/input/raft-pytorch/raft-sintel.pth'
# weights_path = '/kaggle/input/raft-pytorch/raft-things.pth'

ckpt = torch.load(weights_path, map_location=device)
raft_model.to(device)
raft_model.load_state_dict(ckpt)

SMALL_RATE = 0.25 # image scaling rate 
STD_TH = 12 # threshold of std of optical flow

In [ ]:
import torch
import torchvision
torch.__version__, torchvision.__version__

In [ ]:
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
def clip_pred(x_min, y_min, width, height):
    x_min = min(max(int(x_min + 0.5), 0), 1279)
    y_min = min(max(int(y_min + 0.5), 0), 719)
    width = int(max(min(1280 - x_min, int(width + 0.5)),1))
    height = int(max(min(720 - y_min, int(height + 0.5)),1))
    return x_min, y_min, width, height
    
    
def tracking_function(tracker, frame_id, bboxes, scores, verbose=False):
    detects = []
    predictions = []
    if len(scores)>0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min, y_min, bbox_width, bbox_height = clip_pred(box[0], box[1], box[2], box[3])            
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    if verbose:
        print(frame_id, tracked_objects)
    to_add_preds = []
    if USE_TRACKING:
        for tobj in tracked_objects:
            bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
            if verbose:
                print("last_detected_frame_id:", last_detected_frame_id)
            if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
                continue
            # Add objects that have no detections on current frame to predictions
            xc, yc = tobj.estimate[0]
            x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
            #exclude those in the edge
            if (x_min + bbox_width >= 1279) or (y_min + bbox_height) >= 719 or (x_min <= 1) or (y_min <= 1):
                continue
            
            score = tobj.last_detection.scores[0]
            x_min, y_min, bbox_width, bbox_height = clip_pred(x_min, y_min, bbox_width, bbox_height)
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
            
    return predictions, tracker

In [ ]:
best_conf_dict = {
    "0208_m6_B_LS02_newGT_clahe": 0.25,
    "0209_m6_B_LS02_newLGBT": 0.1,
    "best_s6_3600_GT":0.3,
    'yolov5-v79-addgt':0.4,
    'r27-0210-yolov5m6-2400': 0.27,
    
    # 0211
    'r27-0210-yolov5m6-2400':0.27,
    "0210_m6_B_LS02_LGBT_newP": 0.2,
    "0209_m6_B_LS02_LGBT_newP_clahe":0.25,
    'GT2_4680': 0.3,
    'GT_transfer_4680':0.3,
    '0209_s6_B_LS02_newGT_img4096':0.35,
    '0212_crcnn_1600':0.25,
}
SAHI_dict = {
    '0203_m6_1200_sliced_S720_ma025_lr5e4': {
        "SH":720,
        "SW":720,
        "OHR":0.2,
        "OWR":0.2,
    }
}
best_iou_dict = {}
best_size_dict = {
    "yolov5-v74-fpimage": 9000,
    'yolov5-v79-addgt':9000,
    'r22-0206-yolov5x6-1920-full':int(1920*1.3),
    'best_s6_3600_GT':int(3600 * 1.3),
    'GT2_4680':4680,
    'GT_transfer_4680':4680,
    '0209_s6_B_LS02_newGT_img4096': int(4096 * 1.3),
    'r27-0210-yolov5m6-2400': 3120,
    'r27_m6_add_gt':3120,
}
ckpt_path_dict = {
    "yolov5-v74-fpimage": "/kaggle/input/yolov5-v74-fpimage/v74/weights/best.pt",    
    "yolov5-v79-addgt": "/kaggle/input/yolov5-v79-addgt/v79/weights/best.pt",    
    'r22-0206-yolov5x6-1920-full': "/kaggle/input/r22-0206-yolov5x6-1920-full/v1/weights/last.pt",
    'best_s6_3600_GT': "/kaggle/input/yolov5-models/best_s6_3600_GT.pt",
    
    'GT2_4680': "/kaggle/input/yolov5-models/last_s6_3600_GT2_all.pt",
    'GT_transfer_4680':  "/kaggle/input/yolov5-models/last_s6_3600_GT_transfer_all.pt",
    
    '0209_s6_B_LS02_newGT_img4096': "/kaggle/input/0209-s6-b-ls02-newgt-img4096/best_all.pt",    
    'r27-0210-yolov5m6-2400': "/kaggle/input/r27-yolov5m6-2400-whole/weights/last.pt",
    'r27_m6_add_gt': "/kaggle/input/r27-yolov5m6-2400-whole/full_add_gt/weights/last.pt",
}
outside_models = ['yolov5-v79-addgt','best_s6_3600_GT','GT2_4680', 'GT_transfer_4680','0209_s6_B_LS02_newGT_img4096','r27-0210-yolov5m6-2400', 'r27_m6_add_gt']

In [ ]:
RUN_TEST = True
AUGMENT = True
IOU = 0.4
IOU_TH=0.4

USE_RAFT = True
USE_TRACKING = True
USE_LOW_CONF = True
USE_MULTI_TRACKER = True
second_WBF_LB = True
PRE_CONF_TH = 0.07
SKIP_BOX_TH = 0.2

LB_model = 'yolov5-v79-addgt'
rcnn_models = ['0212_crcnn_1600_whole']

In [ ]:
meta_params = {}
# 4 models + LB
model_versions = ['GT2_4680', 'r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096', '0209_m6_B_LS02_LGBT_newP_clahe_whole', 'yolov5-v79-addgt']
# 5 yolo + 1 rcnn
# model_versions = ['GT2_4680', 'r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096', '0209_m6_B_LS02_LGBT_newP_clahe_whole', "GT_transfer_4680", 
#                   '0212_crcnn_1600_whole']
# 7 yolo
# model_versions = ['GT2_4680', 'r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096', '0209_m6_B_LS02_LGBT_newP_clahe_whole', "GT_transfer_4680", 
#                   '0210_m6_B_LS02_LGBT_newP_whole', '0208_m6_B_LS02_newGT_clahe_whole']

# '0210_m6_B_LS02_LGBT_newP_whole',
#                 '0209_m6_B_LS02_LGBT_newP_clahe_whole', 'r27_m6_add_gt']
#                   ,'0210_m6_B_LS02_LGBT_newP_whole', "0209_m6_B_LS02_LGBT_newP_clahe_whole", 
#                   'best_s6_3600_GT','0209_m6_B_LS02_newLGBT_whole']
#model_versions = ['0211_crcnn_1600']
for model_version in model_versions:
    if model_version in outside_models:
        # from others dataset
        meta_params[model_version] = {"ckpt_path": ckpt_path_dict[model_version]}
        continue
    
    model_version_k = "-".join(model_version.lower().split("_"))
    model_folder = Path(f"/kaggle/input/{model_version_k}/")
    params_path = model_folder / "config" / "params.yaml"
    with open(params_path) as file:
        params = yaml.load(file, Loader=yaml.FullLoader)    
    for key, val in params.items():
        if "dir" in key or "path" in key or "file" in key:
            params[key] = Path(val)         
    meta_params[model_version] = params

In [ ]:
home_dir = Path("../input/")
data_dir = Path("../input/tensorflow-great-barrier-reef")
def transform_params(version, params, home_dir, data_dir):
    version_k = "-".join(version.lower().split("_"))
    for key, val in params.items():
        if "path" in key or "dir" in key:
            old_val = val.resolve()
            if key not in ["data_path", "root_dir"]:
                new_val = home_dir / "/".join(str(old_val).split("/")[6:])
            else:
                new_val = data_dir
            if version in str(new_val):
                if key != "ckpt_path":
                    new_val = Path(str(new_val).replace(version, version_k))
                else:
                    if params['tools'] == 'mmdetection':
                        ckp = home_dir / version_k
                        if "whole" in version:
                            ckp = glob(str(ckp) + "/" + "epoch*.pth")[0]
                        else:
                            ckp = glob(str(ckp) + "/" + "best*.pth")[0]
                        new_val = Path(ckp)
                    else:
                        # YOLOv5
                        new_val = home_dir / f"{version_k}/TGBR/{version}/weights/last.pt"
                    
            params[key] = new_val
    return params

In [ ]:
for model_version, params in meta_params.items():
    params["repo"] = '/kaggle/input/yolov5-vincent/yolov5'
    old_model_version = model_version
    if model_version[-6:] == "_whole": 
        model_version = model_version[:-6]
    
    params["conf"] = PRE_CONF_TH if (USE_LOW_CONF and model_version != LB_model) else best_conf_dict.get(model_version, 0.2) 
    params["iou"] = best_iou_dict.get(model_version, IOU)
for model_version in model_versions:
    if model_version in outside_models:
        continue
    params = meta_params[model_version]
    params = transform_params(model_version, params, home_dir, data_dir)
    meta_params[model_version] = params
clahe = cv2.createCLAHE(clipLimit=5, tileGridSize=(10, 10))

In [ ]:
from ensemble_boxes import weighted_boxes_fusion, nms, non_maximum_weighted, soft_nms

def modify_size(boxes, to01=True):
    if len(boxes) > 0:
        if to01:
            boxes[:,2:] = boxes[:,2:] + boxes[:,:2]
            boxes[:,0] = boxes[:,0] / 1279.
            boxes[:,1] = boxes[:,1] / 719.
            boxes[:,2] = boxes[:,2] / 1279.
            boxes[:,3] = boxes[:,3] / 719.
        else:
            boxes[:,0] = boxes[:,0] * 1279.
            boxes[:,1] = boxes[:,1] * 719.
            boxes[:,2] = boxes[:,2] * 1279.
            boxes[:,3] = boxes[:,3] * 719.
            boxes[:,2:] = boxes[:,2:] - boxes[:,:2]
        
    return boxes
    
def run_wbf(bboxes, confs, iou_thr=0.5, skip_box_thr=0.1, verbose=False, weights=None):
    if len(bboxes) == 1:
        return bboxes[0], confs[0], []   
    else:
        for i in range(len(bboxes)):
            bboxes[i] = modify_size(bboxes[i], to01=True)    
        
    labels = [np.ones(len(conf)) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(bboxes, confs, labels, iou_thr=iou_thr, skip_box_thr=0.001, allows_overflow=True, conf_type='avg', weights=weights)
    
    boxes = modify_size(boxes, to01=False)  
    boxes = [box for i,box in enumerate(boxes) if scores[i] >= skip_box_thr]
    scores = [conf for conf in scores if conf >= skip_box_thr]

    return boxes, scores, labels

In [ ]:
def load_model(params):
    try:
        model = torch.hub.load(params['repo'],
                               'custom',
                               path=params['ckpt_path'],
                               source='local',
                               force_reload=True)  # local repo
    except:
        print("torch.hub.load failed, try torch.load")
        model = torch.load(params['ckpt_path'])
    model.conf = params["conf"]  # NMS confidence threshold
    model.iou  = params["iou"]  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 50  # maximum number of detections per image
    return model

In [ ]:
from PIL import Image
#colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255)) for idx in range(1)]
colors = [(192, 173, 39)]
def show_img(img, bboxes, colors, bbox_format='yolo', labels=None):
    names  = ['cots']*len(bboxes) if labels is None else ["pred"] * (len(labels) - sum(labels)) + ["real"] * sum(labels)
    labels = [0]*len(bboxes) if labels is None else labels
    img    = util.draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
if first:
    import greatbarrierreef
    env = greatbarrierreef.make_env()# initialize the environment
    iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission
    first = False

In [ ]:
models = {}
for model_version, params in meta_params.items():
    print(model_version)
    if params.get('tools',"yolov5") == 'mmdetection':
        test_pipeline = [
            mmcv.utils.config.ConfigDict(type='LoadImageFromFile'),
            mmcv.utils.config.ConfigDict(
                type='MultiScaleFlipAug',
                img_scale=[(params['img_size'], params['img_size'])] ,
                flip=[False],
                transforms=[
                    mmcv.utils.config.ConfigDict(type='Resize', keep_ratio=False),
                    mmcv.utils.config.ConfigDict(type='Pad', size_divisor=32),
                    mmcv.utils.config.ConfigDict(type='RandomFlip', direction='horizontal'),
                    mmcv.utils.config.ConfigDict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    mmcv.utils.config.ConfigDict(type='ImageToTensor', keys=['img']),
                    mmcv.utils.config.ConfigDict(type='Collect', keys=['img'])
                ])
        ]

        mm_config =  mmcv.Config.fromfile(str(params['cfg_dir'] / "config.py"))
        mm_config['data']['test']['pipeline'] = test_pipeline
        model = init_detector(mm_config, str(params['ckpt_path']), device='cuda:0')            
    elif "slice" in model_version:
        model = Yolov5DetectionModel(
            model_path=str(params['ckpt_path']),
            confidence_threshold=params["conf"],
            image_size = int(params['img_size'] * 1.3),
            device="cuda:0", # or 'cuda:0'
        )    
    else:            
        model = load_model(params)
        device = torch.device("cuda")
        model.to(device)
    models[model_version] = model    

In [ ]:
df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
np.random.seed(13)
candidates = df.sample(10)

In [ ]:
iter_check = []
for idx,row in candidates.iterrows():
    path = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{}/{}.jpg".format(row['video_id'], row['video_frame'])
    img = cv2.imread(path)[...,::-1]
    iter_check.append(img)

In [ ]:
def init_tracker():
    return Tracker(
                distance_function=euclidean_distance, 
                distance_threshold=30,
                hit_inertia_min=3,
                hit_inertia_max=6,
                initialization_delay=2,
            )

def pred_img(img, tracker, frame_id, idx, verbose=False, show_img_num=10, raft_image_old=None):
    # RAFT for sequence change detection    
    if USE_MULTI_TRACKER:
        trackers = tracker
        
    raft_image = None
    if USE_RAFT:
        raft_image = cv2.resize(img, dsize=None, fx=SMALL_RATE, fy=SMALL_RATE)
        raft_image = torch.from_numpy(raft_image).permute(2, 0, 1).float().to(device)

        if raft_image_old is not None:
            img1 = raft_image[None].to(device)
            img2 = raft_image_old[None].to(device)

            padder = InputPadder(img1.shape)
            img1, img2 = padder.pad(img1, img2)
            with torch.no_grad():
                flow_low, flow_up = raft_model(img1, img2, iters=10, test_mode=True)
            flow_std = np.std(flow_up.cpu().numpy())
            if verbose:
                print("flow_std {:.4f}".format(flow_std))

            if flow_std > STD_TH: # reinitialize tracker
                if verbose:
                    print("flow_std {:.4f} > STD_TH {}, reinitialize tracker ".format(flow_std, STD_TH))
                if USE_MULTI_TRACKER:
                    for j in trackers.keys():
                        trackers[j] = init_tracker()
                else:
                    tracker = init_tracker()
                frame_id = 0        
    
    prd_bboxs_all = []
    confis_all = []
    
    img_clahe = img.copy()
    for i in range(3):
        img_clahe[:, :, i] = clahe.apply((img_clahe[:, :, i])) 
        
    LB_bboxes = []
    LB_confs = []
    second_bbox = []
    seconf_conf = []
    for j, (model_version, model) in enumerate(models.items()):
        CONF = PRE_CONF_TH if USE_LOW_CONF else meta_params[model_version]['conf']
        if verbose:
            print(model_version)
        if model_version in best_size_dict:
            SIZE = best_size_dict[model_version]
        else:
            SIZE = int(meta_params[model_version]["img_size"] * 1.3)
            
        if meta_params[model_version].get("tools","yolov5") == "mmdetection":
            if meta_params[model_version].get("use_clahe", False):
                result = inference_detector(model, img_clahe[...,::-1])
            else:
                result = inference_detector(model, img[...,::-1])
                
            bboxes = result[0][:,:4]
            bboxes[:,2:] = bboxes[:,2:] - bboxes[:,:2]
            confs = result[0][:,4]
            bboxes = [b for i,b in enumerate(bboxes) if confs[i] >= CONF]
            confs = [c for c in confs if c >= CONF]
        elif "slice" in model_version:
            if model_version[-6:] == "_whole": 
                model_version = model_version[:-6]
            if meta_params[model_version].get("use_clahe", False):
                result_sliced = get_sliced_prediction(
                    img_clahe,
                    model,
                    slice_height = SAHI_dict[model_version]['SH'],
                    slice_width = SAHI_dict[model_version]['SW'],
                    overlap_height_ratio = SAHI_dict[model_version]['OHR'],
                    overlap_width_ratio = SAHI_dict[model_version]['OWR'],
                    postprocess_match_threshold = IOU)
            else:
                result_sliced = get_sliced_prediction(
                    img,
                    model,
                    slice_height = SAHI_dict[model_version]['SH'],
                    slice_width = SAHI_dict[model_version]['SW'],
                    overlap_height_ratio = SAHI_dict[model_version]['OHR'],
                    overlap_width_ratio = SAHI_dict[model_version]['OWR'],
                    postprocess_match_threshold = IOU)

            object_prediction_list = result_sliced.object_prediction_list
            confs = [obj_pred.score.value for obj_pred in object_prediction_list]
            bboxes = np.array([obj_pred.bbox.to_coco_bbox() for obj_pred in object_prediction_list])                    
        else:                
            if meta_params[model_version].get("use_clahe", False):
                bboxes, confs = util.predict(model, img_clahe, size=SIZE, augment=AUGMENT)
            else:
                bboxes, confs = util.predict(model, img, size=SIZE, augment=AUGMENT)
                    
        bboxes = np.array(bboxes, dtype=np.float64)
        confs = np.array(confs)
        if USE_MULTI_TRACKER:
            if verbose:
                print("Use multi tracker")
            predictions, tracker = tracking_function(trackers[j], frame_id, bboxes, confs, 0)
            trackers[j] = tracker
            bboxes =  np.array([[float(p) for p in pred.split(" ")[1:]] for pred in predictions])
            confs =  np.array([float(pred.split(" ")[0]) for pred in predictions])
        
        if model_version in rcnn_models:
            second_bbox = bboxes
            second_conf = confs # [c * 0.8 for c in confis]
        elif second_WBF_LB and model_version == LB_model:
            LB_bboxes = np.array([bboxes[i] for i, c in enumerate(confs) if c >= meta_params[model_version]['conf']])
            LB_confs = np.array([c for i, c in enumerate(confs) if c >= meta_params[model_version]['conf']])
        else:
            prd_bboxs_all.append(bboxes)
            confis_all.append(confs)
    # wbf
    wbf_boxes, wbf_confs, _ = run_wbf(prd_bboxs_all, confis_all, IOU_TH, SKIP_BOX_TH, verbose)
    if verbose:
        print("original WBF prediction")
        for i, c in enumerate(wbf_confs):
            print(i, c, wbf_boxes[i])
    if len(second_bbox) > 0 and len(wbf_boxes) > 0:
        keep_second_part = util.calc_iou(np.array(second_bbox), np.array(wbf_boxes)).max(axis=1) >= 0.5
        second_bbox = second_bbox[keep_second_part]
        second_conf = (np.array(second_conf)[keep_second_part]).tolist()
        if len(second_bbox) > 0:
            if verbose:
                print("consider {} second bbox and {} wbf_bbox".format(len(second_bbox), len(wbf_boxes)))
            wbf_boxes, wbf_confs, _ = run_wbf([np.array(wbf_boxes)] +[second_bbox], [wbf_confs] + [second_conf], IOU_TH, 0, 
                                              verbose=False,weights=[len(model_versions)-1, 1])            
    
    if second_WBF_LB and len(LB_bboxes) > 0:
        if verbose:
            print("second_WBF_LB, add {} LB predictions: {} with confs {}".format(len(LB_bboxes), LB_bboxes, LB_confs))
            
        wbf_boxes, wbf_confs, _ = run_wbf([np.array(wbf_boxes)] +[LB_bboxes], [wbf_confs] + [LB_confs.tolist()], IOU_TH, 0, 
                                          verbose=False,weights=[len(model_versions)-1, 1])            
    # tracking
    if not USE_MULTI_TRACKER:
        predictions, tracker = tracking_function(tracker, frame_id, wbf_boxes, wbf_confs, verbose)
    else:
        predictions = ['{:.2f} {} {} {} {}'.format(wbf_confs[i], b[0], b[1], b[2], b[3]) for i, b in enumerate(wbf_boxes)]
    if verbose:
        print(wbf_boxes)
        print(" ".join(predictions))
        
    if idx < show_img_num:
        display(util.show_img(img, wbf_boxes, colors, bbox_format='coco'))    
    if USE_MULTI_TRACKER:    
        return predictions, trackers, raft_image
    else:
        return predictions, tracker, raft_image

In [ ]:
if USE_MULTI_TRACKER:
    trackers = {j: init_tracker() for j in range(len(models))}
else:
    tracker = init_tracker()
frame_id = 0
raft_image_old = None
for idx, img in enumerate(tqdm(iter_check)):
    if USE_MULTI_TRACKER:
        predictions, trackers, raft_image_old = pred_img(img, trackers, frame_id, idx, 
                                                         verbose=True, show_img_num=20, 
                                                         raft_image_old=raft_image_old)
    else:
        predictions, tracker, raft_image_old = pred_img(img, tracker, frame_id, idx, 
                                                        verbose=True, show_img_num=20, 
                                                        raft_image_old=raft_image_old)
        
    frame_id += 1

In [ ]:
if USE_MULTI_TRACKER:
    trackers = {j: init_tracker() for j in range(len(models))}
else:
    tracker = init_tracker()

frame_id = 0
raft_image_old = None
if RUN_TEST:
    print("Run test")    
    for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
        if USE_MULTI_TRACKER:
            predictions, trackers, raft_image_old = pred_img(img, trackers, frame_id, idx, 
                                                             verbose=(idx<=2), show_img_num=3, 
                                                             raft_image_old=raft_image_old)
        else:
            predictions, tracker, raft_image_old = pred_img(img, tracker, frame_id, idx, 
                                                            verbose=(idx<=2), show_img_num=3, 
                                                            raft_image_old=raft_image_old)
        pred_df['annotations'] = " ".join(predictions)
        env.predict(pred_df)
        frame_id += 1
    sub_df = pd.read_csv('submission.csv')
    print(sub_df.head())

In [ ]:
if use_mm:
    import shutil
    shutil.rmtree('/kaggle/working/mmdetection')

In [ ]:
'''
best_conf_dict= {
    '0128_yolov5l_v3': 0.25,
    "0128_yolov5l_v3_highFP": 0.25,
    "0129_yolov5l_v3_highFP_1800": 0.25,
    "0129_yolov5l_v3_highFP_1800_e11": 0.16,
    "0129_yolov5l_v3_highFP_clahe": 0.19,
    '0129_yolov5l_v3_highFP_3600_e11': 0.25,
    '0129_yolov5l_v3_1800_clahe': 0.15,
    '0129_yolov5l_v3_2700_clahe': 0.2,
    '0130_yolov5m6_2300':0.27,
    "yolov5-v74-fpimage": 0.4,
    '0130_yolov5s6_3100': 0.31,
    '0131_yolov5s6_3100': 0.31,
    "0130_yolov5m6_noHighFP_2300": .40,
    "0130_yolov5m6_noClahe_2300": .23,
    "0131_yolov5m6_noClahe_2300": .23,
    "0130_yolov5m6_2300_sheep": .11,
    "0131_yolov5m6_2300_sheep": .11,
    '0202_frcnn_1600_highFP_e8': 0.34,
    '0203_m6_1200_sliced_S720_ma025_lr5e4' : 0.53, # 0.44 for no AUG
    "0205_m_TPH_1900_B": 0.27,
    '0204_yolov5s6_B': 0.20,
    '0206_yolov5s6_B': 0.20,
    '0205_yolov5m6_B': 0.17,
    '0206_yolov5m6_B_LS02': 0.3,
    'r22-0206-yolov5x6-1920-full':0.2,
    '0207_yolov5m6_B_LS02_newGT': 0.38, # CV 696
    '0207_yolov5m6_B_LS02_newGT_clahe':0.33, # 696
}

'''